### Install and import dependencies

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

### Instantiate Model

In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

### Encode and Calculate Sentiment

In [3]:
tokens = tokenizer.encode('I am feeling happy but not that much', return_tensors='pt')

In [4]:
tokens

tensor([[  101,   151, 10345, 35260, 19308, 10502, 10497, 10203, 12977,   102]])

In [5]:
result = model(tokens)

In [6]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-1.7688, -0.0713,  1.6600,  1.0543, -0.8139]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [7]:
result.logits

tensor([[-1.7688, -0.0713,  1.6600,  1.0543, -0.8139]],
       grad_fn=<AddmmBackward0>)

In [8]:
# Sentiment rating 1-5
int(torch.argmax(result.logits))+1

3

### Collect reviews 

In [9]:
r = requests.get('https://www.tripadvisor.com/Attraction_Review-g295424-d1210327-Reviews-The_Dubai_Mall-Dubai_Emirate_of_Dubai.html')

In [10]:
soup = BeautifulSoup(r.text, 'html.parser')

In [45]:
r.text

'<!DOCTYPE html><html lang="en-US"><head><meta http-equiv="content-type" content="text/html; charset=utf-8"/><meta http-equiv="content-language" content="en"/><link href="https://static.tacdn.com/css2/webfonts/TripSans/TripSans-VF.woff2?v1.002" rel="preload" as="font" type="font/woff2" crossorigin="anonymous"/><link rel="icon" id="favicon" href="https://static.tacdn.com/favicon.ico?v2" type="image/x-icon"/><link rel="mask-icon" sizes="any" href="https://static.tacdn.com/img2/brand_refresh/application_icons/mask-icon.svg" color="#000000"/><meta name="theme-color" content="#34e0a1"/><meta name="format-detection" content="telephone=no"/><meta property="al:ios:app_name" content="TripAdvisor"/><meta property="al:ios:app_store_id" content="284876795"/><meta property="twitter:app:id:ipad" name="twitter:app:id:ipad" content="284876795"/><meta property="twitter:app:id:iphone" name="twitter:app:id:iphone" content="284876795"/><meta property="al:ios:url" content="tripadvisor://www.tripadvisor.com

In [53]:
regex = re.compile('.*yCeTE.*')
results = soup.find_all('span', {'class':regex})

In [54]:
results[0]

<span class="yCeTE">An architectural wonder</span>

In [55]:
reviews = [result.text for result in results]

In [56]:
reviews[0]

'An architectural wonder'

### Load Reviews into DataFrame and Score

In [57]:
import numpy as np
import pandas as pd

In [58]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [59]:
df.head()

,review
0,An architectural wonder
1,Massive would be an understatement. A colossal...
2,Great place to hangout
3,Love the mall and always visit the fountain to...
4,Buy Google Voice Accounts


In [60]:
df.shape

(20, 1)

In [61]:
df['review'].iloc[0]

'An architectural wonder'

In [62]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [63]:
sentiment_score(df['review'].iloc[1])

3

In [64]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [67]:
df.head(20)

,review,sentiment
0,An architectural wonder,5
1,Massive would be an understatement. A colossal...,4
2,Great place to hangout,5
3,Love the mall and always visit the fountain to...,5
4,Buy Google Voice Accounts,5
5,Buy Google Voice AccountIs it true that you ar...,5
6,Not worth the journey,1
7,If you like shopping indoors on holiday and pa...,3
8,Shop 'Til You Drop (And Then Some) at Dubai Mall,1
9,"As a real estate agency, we often recommend Du...",5


In [66]:
df['review'].iloc[3]

'Love the mall and always visit the fountain to catch the show when I’m in Dubai. The show lasts All of 5 minutes and is quite an interesting display of light and sound. The mall has many restaurants to choose from and an amazing variety of stores - fromHigh- end to midrange. It’s massive so be prepared to walk. Parking can also be a challenge on weekends and valet is available at a cost (yes it’s quite steep), so I would recommend an Uber. Something else I love about the mall is the aquarium'